## Coding Exercise #0509

In [1]:
import nltk
from numpy.random import randint, seed
from sklearn.feature_extraction.text import CountVectorizer

### 1. n-Gram based autofill:

In [2]:
!pip install EbookLib


In [3]:
!pip install ebooklib
!pip install beautifulsoup4

In [6]:
from ebooklib import epub
from bs4 import BeautifulSoup
import ebooklib

def extraer_texto_epub(ruta):
    libro = epub.read_epub(ruta)
    texto_total = ''
    for item in libro.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            contenido = item.get_content()
            soup = BeautifulSoup(contenido, 'html.parser')
            texto_total += soup.get_text()
    return texto_total


In [7]:
text = extraer_texto_epub('elegi_vivir.epub')

In [8]:
text

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \nÍndice\nCubierta\nAGRADECIMIENTOS\nPRÓLOGO\nANTES DEL ACCIDENTE\nEL ACCIDENTE\nEL RESCATE\nLOS RECUERDOS DE UN AMIGO: DIEGO ZANOLLI\nEL DESPERTAR\nEN LA UCI\nLOS PRIMEROS DÍAS EN LA CLÍNICA\nTONELADAS DE CARIÑO\nEL COMIENZO DE MI REHABILITACIÓN\nLA CUENTA REGRESIVA\nEL TESTIMONIO DE LEONOR\nPREPARANDO EL PRIMER VIAJE\nLA LLEGADA A FILADELFIA Y A MOSS REHAB INSTITUTE\nEL INICIO DE LA TERAPIA FÍSICA\nLA TERAPIA OCUPACIONAL Y LA ACEPTACIÓN\nLA MOTIVADORA TERAPIA RECREACIONAL\nDE REGRESO A CHILE\nOTRA VEZ EN CASA\nPREPARANDO MI RETORNO A CLASES\nOTRA VEZ A MOSS\nEL ARCO IRIS DESPUÉS DE LA LLUVIA\nOTRO VIAJE TERMINA\nRETOMANDO MIS ESTUDIOS\nLA TELETÓN\nMI VIDA CONTINÚA\nEPÍLOGO\nCUADERNO DE IMÁGENES\nCréditos\n\n\n\n\n\n\n\n\n \nA mi mamá, por su entrega incondicional. \nA mi familia y amigos, por su cariño y apoyo. \nY a Ricardo, por siempre creer en mí. \n\n\n\n\n\n\n\n\nAGRADECIMIENTOS \n \nNo fue fácil decidirme a publicar este libro. Para mí es mucho

In [9]:
import re

def limpiar_texto(texto):
    texto = texto.lower()  # pasar todo a minúsculas
    texto = re.sub(r'\s+', ' ', texto)  # eliminar saltos de línea y espacios múltiples
    texto = re.sub(r'[“”‘’]', '', texto)  # comillas raras
    texto = texto.replace('\xa0', ' ')  # eliminar caracteres extraños
    return texto.strip()

corpus_limpio = limpiar_texto(text)


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Crear el tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts([corpus_limpio])

total_palabras = len(tokenizer.word_index) + 1
print(f"Total de palabras únicas: {total_palabras}")

# Crear secuencias de entrenamiento
input_sequences = []
palabras = corpus_limpio.split()

for i in range(1, len(palabras)):
    secuencia = palabras[:i+1]
    codificada = tokenizer.texts_to_sequences([' '.join(secuencia)])[0]
    input_sequences.append(codificada)

# Pad sequences
secuencias_padded = pad_sequences(input_sequences, padding='pre')

# Separar en X e y
X = secuencias_padded[:, :-1]
y = secuencias_padded[:, -1]

# One-hot encode y
from tensorflow.keras.utils import to_categorical
y = to_categorical(y, num_classes=total_palabras)


Total de palabras únicas: 7856


#### 1.1. n-Gram trial run:

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk

In [11]:
n = 3                                                            # Can be changed to a number equal or larger than 2.
n_min = n
n_max = n
n_gram_type = 'word'                                             # n-Gram with words.
vectorizer = CountVectorizer(ngram_range=(n_min, n_max), analyzer=n_gram_type)

In [12]:
n_grams = vectorizer.fit([corpus_limpio]).get_feature_names_out()            # Get the n-Grams as a list.
n_gram_cts = vectorizer.transform([corpus_limpio]).toarray()             #  The output is an array of array.
n_gram_cts = list(n_gram_cts[0])                                 # Convert into a simple list.

In [13]:
list(zip(n_grams,n_gram_cts))                                    # Make a list of tuples and show.

[('11 de agosto', np.int64(1)),
 ('17 de diciembre', np.int64(1)),
 ('1999 entré estudiar', np.int64(1)),
 ('2002 empezó temprano', np.int64(1)),
 ('2002 fue especialmente', np.int64(1)),
 ('2002 fue un', np.int64(1)),
 ('2003 antes del', np.int64(1)),
 ('2003 estaba programada', np.int64(1)),
 ('2003 llegué por', np.int64(1)),
 ('2003 mis papás', np.int64(1)),
 ('2003 sino también', np.int64(1)),
 ('2004 daniela garcía', np.int64(1)),
 ('2004 estas fotos', np.int64(1)),
 ('2004 fui nuevamente', np.int64(1)),
 ('2014 2004 daniela', np.int64(1)),
 ('2014 penguin random', np.int64(1)),
 ('21 22 de', np.int64(1)),
 ('22 años no', np.int64(1)),
 ('22 de noviembre', np.int64(2)),
 ('280 piso santiago', np.int64(1)),
 ('30 de octubre', np.int64(8)),
 ('50 con vagones', np.int64(1)),
 ('9789562584005 conversión formato', np.int64(1)),
 ('abajo como siempre', np.int64(1)),
 ('abajo tratando de', np.int64(1)),
 ('abandonaban incluso intenté', np.int64(1)),
 ('abandonaban jamás iba', np.int64(1)

#### 1.2. Train by making a dictionary based on n-Grams:

In [14]:
n = 3                                                           # Can be changed to a number equal or larger than 2.
n_min = n
n_max = n
n_gram_type = 'word'
vectorizer = CountVectorizer(ngram_range=(n_min,n_max), analyzer = n_gram_type)

In [15]:
nltk.download('punkt_tab')

n_grams = vectorizer.fit([corpus_limpio]).get_feature_names_out()           # A list of n-Grams.
my_dict = {}
for a_gram in n_grams:
    words = nltk.word_tokenize(a_gram)
    a_nm1_gram = ' '.join(words[0:n-1])  # El (n-1)-grama
    next_word = words[-1]               # Palabra siguiente
    if a_nm1_gram not in my_dict:
        my_dict[a_nm1_gram] = [next_word]
    else:
        my_dict[a_nm1_gram].append(next_word)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [16]:
# Ver parte del diccionario
for k in list(my_dict)[:10]:
    print(f"{k} ➜ {my_dict[k]}")

11 de ➜ ['agosto']
17 de ➜ ['diciembre']
1999 entré ➜ ['estudiar']
2002 empezó ➜ ['temprano']
2002 fue ➜ ['especialmente', 'un']
2003 antes ➜ ['del']
2003 estaba ➜ ['programada']
2003 llegué ➜ ['por']
2003 mis ➜ ['papás']
2003 sino ➜ ['también']


In [17]:
# View the dictionary.
my_dict

{'11 de': ['agosto'],
 '17 de': ['diciembre'],
 '1999 entré': ['estudiar'],
 '2002 empezó': ['temprano'],
 '2002 fue': ['especialmente', 'un'],
 '2003 antes': ['del'],
 '2003 estaba': ['programada'],
 '2003 llegué': ['por'],
 '2003 mis': ['papás'],
 '2003 sino': ['también'],
 '2004 daniela': ['garcía'],
 '2004 estas': ['fotos'],
 '2004 fui': ['nuevamente'],
 '2014 2004': ['daniela'],
 '2014 penguin': ['random'],
 '21 22': ['de'],
 '22 años': ['no'],
 '22 de': ['noviembre'],
 '280 piso': ['santiago'],
 '30 de': ['octubre'],
 '50 con': ['vagones'],
 '9789562584005 conversión': ['formato'],
 'abajo como': ['siempre'],
 'abajo tratando': ['de'],
 'abandonaban incluso': ['intenté'],
 'abandonaban jamás': ['iba'],
 'abandonando mi': ['cuerpo'],
 'abandonar el': ['proscenio'],
 'abandonar la': ['casa'],
 'abandonaría la': ['terapia'],
 'abandonó la': ['uci'],
 'abarrotada no': ['dejaba'],
 'abdominales en': ['distintas'],
 'abdominales levantar': ['algunos'],
 'abierta lo': ['que'],
 'abierta

#### 1.3. Predict the next word:

In [29]:
input_str = 'por josé'
print(input_str in my_dict)


True


In [20]:
from random import randint, seed

In [21]:
# Helper function that picks the following word.
def predict_next(a_nm1_gram):
    if a_nm1_gram not in my_dict:
        return '[desconocido]'  # opcional: puedes retornar None o un mensaje
    value_list_size = len(my_dict[a_nm1_gram])
    i_pick = randint(0, value_list_size - 1)  # importante: -1 para que esté dentro del rango
    return my_dict[a_nm1_gram][i_pick]                # Return the randomly chosen next word.

In [30]:
# Test 1: Predecir una palabra aleatoria
# asegúrate de usar un (n-1)-grama válido
print(predict_next(input_str))

luis


In [31]:
# Another test.
# Ver repeticiones para estimar probabilidad
for i in range(10):
    print(predict_next(input_str))

luis
luis
luis
luis
luis
luis
luis
luis
luis
luis


#### 1.4. Predict a sequence:

In [24]:
# Initialize the random seed.
seed(123)

In [25]:
my_seed_str = 'al verme'
print(my_seed_str in my_dict)

True


In [26]:
a_nm1_gram = my_seed_str
output_string = my_seed_str                                         # Initialize the output string.
while a_nm1_gram in my_dict:
    next_word = predict_next(a_nm1_gram)
    output_string += " " + next_word
    words = nltk.word_tokenize(output_string)
    a_nm1_gram = ' '.join(words[-n+1:])  # actualiza el (n-1)-grama con las últimas n-1 palabras                           # Update a_nm1_gram.

In [27]:
# Output the predicted sequence.

print("Texto generado:")
print(output_string)

Texto generado:
al verme el cariño agradecimiento hacia esas luces blancas rojas qué ocurría me paré asustada fui hacia la casa central maca ya voy llegando le avisé por teléfono con ellos observé que muchos terapeutas trabajaban con sus cuatro hijos tomó una ducha se metió la cama sacudí mis nubarrones mentales decidí que había recuperado caminaba solamente con la hora jamás lograría llegar tiempo al metro donde nos juntaríamos un grupo cohesionado estaban juntos desde primer año cómo iba lograrlo qué vergüenza me daban fuerza ánimo para continuar adelante al carro donde viajaba el resto fuimos ubicados en otro muy al final de pronto advirtió que sus perseguidores pasaban de largo estaba salvado así el rey no le importó manejar siempre ha sido una buena decisión el doctor enrique oyarzún para entrar al parto estoy segura de que sólo era un medio seguro terminada la hora jamás lograría llegar tiempo al metro tampoco quería depender de otros años créeme se pasa increíble después nos pre